In [1]:
import numpy
import pandas
import matplotlib.pyplot as plt
import sys

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.callbacks import EarlyStopping
from datetime import datetime as dt
import time
import math
# 2018/3/3 Kitamura Add Start
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
# 2018/3/3 Kitamura Add End

#スタート時間を保持
starttime = time.time()

# 学習の設定
l_of_s         = 10
n_next         = 3
in_out_neurons = 1
hidden_neurons = 200
batch_size = 10

# 学習用データを抽出する関数
def _load_data(data, n_prev=10, n_next=1, flag=True):
    docX, docY = [], []
    for i in range(n_prev, len(data)-n_next):
        #起点の箇所からn_prevだけ戻った日数分をデータとする
        docX.append(data.iloc[i-n_prev:i].as_matrix())
        #起点の翌日からn_next日数分進んだデータのmax or minをyデータとする
        if flag == True:
            docY.append(max(data.iloc[i+1:i+n_next].as_matrix()))
        else:
            docY.append(min(data.iloc[i+1:i+n_next].as_matrix()))

    alsX = numpy.array(docX)
    alsY = numpy.array(docY)
    alsX
    alsY
    return alsX, alsY

def lstm_model(activation="relu", Optimizer=RMSprop, out_dim=1, hidden_dim=256, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(hidden_dim, 
          batch_input_shape=(None, l_of_s, out_dim),
          return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(out_dim))
    model.add(Activation(activation))
    optimizer = Optimizer(lr=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    return model

def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result


Using TensorFlow backend.


In [2]:
import urllib.request

#ダウンロードする通貨ペア
#https://stooq.com/q/d/?s=usdjpy
#ついでにsekjpy、nokjpy、mxnjpy、sgdjpyとかもありますが、
#2000/01/01からデータがない（あるけど、しばらくopen/high/low/closeがすべて一緒)ので
#ここには７通貨ペアしか記載しておりません。日付とか調整して通貨ペアを増やしてみても
#いいと思います
currency_pair = 'usdjpy'
#currency_pair = 'eurjpy'
#currency_pair = 'gbpjpy'
#currency_pair = 'audjpy'
#currency_pair = 'cadjpy'
#currency_pair = 'chfjpy'
#currency_pair = 'nzdjpy'
#currency_pair = 'sekjpy'
#currency_pair = 'nokjpy'

#スタート日付
start_day     = "20150101"
#終了日を今日に指定
url           = "https://stooq.com/q/d/l/?s=" + currency_pair + \
                "&d1=" + start_day + "&d2=" + dt.today().strftime("%Y%m%d") + "&i=d"
file_name     = currency_pair + '_d.csv'
#取得して、ファイルに保存(よくよく考えると保存しなくてもいいな)
urllib.request.urlretrieve(url, file_name)


('usdjpy_d.csv', <http.client.HTTPMessage at 0x23017e68208>)

In [3]:
# FXデータの読み込み
data = None
data = pandas.read_csv(file_name)
data.columns = ['date', 'open', 'high', 'low', 'close']
data['date'] = pandas.to_datetime(data['date'], format='%Y-%m-%d')

#train開始終了、test開始終了日を設定
#データは2001/01/01からとっていますが、さかのぼって調査する関係上
#20001/01/01からにしています。また買い目対象の調査としてtest_end_dayよりも
#後の日も参照するためtest_end_dayの取得するデータにはしないでください。
#trainの終了日とtestの開始日が連続していますが、連続させなくても動作するか
#検証していません。注意してください
train_start_day   = dt.strptime('2016-01-01', '%Y-%m-%d')
train_end_day     = dt.strptime('2016-12-31', '%Y-%m-%d')
test_start_day    = dt.strptime('2017-01-01', '%Y-%m-%d')
test_end_day      = dt.strptime('2017-12-31', '%Y-%m-%d')
train_start_count = -1
train_end_count   = -1
test_start_count  = -1
test_end_count    = -1

#train/testの開始終了日の配列の場所を調査
for i in range(len(data['date'])):
    if train_start_count == -1 and data['date'][i] >= train_start_day:
        train_start_count = i
    if train_end_count == -1 and data['date'][i] >= train_end_day:
        train_end_count = i
    if test_start_count == -1 and data['date'][i] >= test_start_day:
        test_start_count = i
    if test_end_count == -1 and data['date'][i] >= test_end_day:
        test_end_count = i
        break

#前にl_of_s日分、後ろにn_next分日数が必要なので
#チェック。足りない場合は中止。これを考慮に入れて
#train/testの開始終了日を設定してください
if train_start_count - l_of_s < 0 or \
    test_end_count + n_next > len(data['date']):
    
    print("data range over")
    sys.exit()
    
print('Train Start: ' + str(train_start_count))
print('Train End  : ' + str(train_end_count))
print('Test  Start: ' + str(test_start_count))
print('Test  End  : ' + str(test_end_count))

up_down           = []
up_count          = 0
down_count        = 0
even_count        = 0
#up/downの割合
check_treshhold   = 0.666
loop_flag         = True

#
check_add_percent = 0.0020
check_percent     = 0.02
if min(data.loc[:, 'low']) > 150:
    check_add_percent = 0.20
    check_percent     = 2.0
elif min(data.loc[:, 'low']) > 30:
    check_add_percent = 0.020
    check_percent     = 0.2
    
#close_open_diff = numpy.array([])
#for i in range(train_start_count, test_end_count):
#    close_open_diff = numpy.append(close_open_diff, numpy.array(data.loc[i-1, 'close'] - data.loc[i, 'open']))
#print('Close-Open Diff:' + str(close_open_diff.mean()) + ' ' + str(close_open_diff.std()))
#up_c_o_diff   = close_open_diff.mean() + close_open_diff.std() * 2
#down_c_o_diff = close_open_diff.mean() - close_open_diff.std() * 2
#c_o_d_remove_count = 0
#for i in range(train_start_count, test_end_count):
#    if data.loc[i-1, 'close'] - data.loc[i, 'open'] >= up_c_o_diff or \
#        data.loc[i-1, 'close'] - data.loc[i, 'open'] <= down_c_o_diff:
#        c_o_d_remove_count += 1
#print("c_o_d_remove_count:" + str(c_o_d_remove_count))

while loop_flag:
    up_count = 0
    down_count = 0
    even_count = 0
    check_percent += check_add_percent
    for i in range(train_start_count, test_end_count):
        #起点の日の翌日のopenの値
        open_value = data.loc[i+1, 'open']
        #起点の日から翌日からn_next日数分のhighの最大値
        max_value = max(data.loc[i+1:i+n_next, 'high'])
        #起点の日から翌日からn_next日数分のlowの最小値
        min_value = min(data.loc[i+1:i+n_next, 'low'])
        #ここは起点の日の翌日のopenの値と起点の日の翌日から
        #n_next日数分のhighの最高値かlowの最小値が規定の値上に
        #差が広がったカウントを調べる。これで上がった・下がったを
        #回数を調べる
        
        if abs(max_value - open_value) >= check_percent and \
            abs(open_value - min_value) < check_percent:
            up_count += 1
        elif abs(open_value - min_value) >= check_percent and \
            abs(max_value - open_value) < check_percent:
            down_count += 1  
        else:
            even_count += 1
    
    #(上がった日+下がった日)/全体の日数でcheck_treshholdを超えたか調べる
    #超えていればその値をベースにする
    print('(U+D)/(U+D+E): ' + str(math.floor((up_count + down_count) / \
                                             (up_count + down_count +even_count) * 100)) + '%')    
    if (up_count + down_count) / (up_count + down_count +even_count) > check_treshhold:
        break

#上がった・下がったの判定するための数字（もっと言うと円がどれだけ動いたか）
print('p            : ' + str(check_percent))
print("UP   COUNT   : " + str(up_count))
print("DOWN COUNT   : " + str(down_count))
print("EVNE COUNT   : " + str(even_count))
print('(U+D)/(U+D+E): ' + str(math.floor((up_count + down_count) / \
                                         (up_count + down_count +even_count) * 100)) + '%')    

#sys.exit()
#上がった・下がったの判定用
#ここではtestの範囲のみ取得
for i in range(test_start_count, test_end_count):
    open_value = data.loc[i+1, 'open']
    close_value = data.loc[i, 'open']
    #起点の日から翌日からn_next日数分のhighの最大値
    max_value = max(data.loc[i+1:i+n_next, 'high'])
    #起点の日から翌日からn_next日数分のlowの最小値
    min_value = min(data.loc[i+1:i+n_next, 'low'])
    #ここは起点の日の翌日のopenの値と起点の日の翌日から
    #n_next日数分のhighの最高値かlowの最小値が規定の値上に
    #差が広がったカウントを調べる。これで上がった・下がったを
    #回数を調べる
    if abs(max_value - open_value) >= check_percent and \
        abs(open_value - min_value) < check_percent:
        up_down.append(1)
    elif abs(open_value - min_value) >= check_percent and \
        abs(max_value - open_value) < check_percent:
        up_down.append(-1)
    else:
        up_down.append(0)

max_value = max(data['high'])
min_value = min(data['low'])
average_value = (max_value+min_value)/2
diff_value = max_value - average_value
print ('max: ' + str(max_value))
print ('min: ' + str(min_value))
print ('ave: ' + str(average_value))
print ('dif: ' + str(diff_value))

#for i in range(len(data['high'].index)):
#data.loc[i, 'high'] = (data.loc[i, 'high'] - average_value) / diff_value
#data.loc[i, 'low']  = (data.loc[i, 'low'] - average_value) / diff_value
#data.loc[i, 'open'] = (data.loc[i, 'open'] - average_value) / diff_value
#data.loc[i, 'close'] = (data.loc[i, 'close'] - average_value) / diff_value

# 2018/3/3 Kitamura add Start
# 正規化
#scaler  = MinMaxScaler( feature_range=(0, 1) )    
#data.loc[i, 'high'] = scaler.fit_transform(data.loc[i, 'high'])
#data.loc[i, 'low'] = scaler.fit_transform(data.loc[i, 'low'])
#data.loc[i, 'open'] = scaler.fit_transform(data.loc[i, 'open'])
#data.loc[i, 'close'] = scaler.fit_transform(data.loc[i, 'close'])
# 2018/3/3 Kitamura add End

data = data.sort_values(by='date')
data = data.reset_index(drop=True)
data = data.loc[:, ['date', 'open','high', 'low', 'close']]

#グラフ化
#plt.plot(data['date'], data['high'])
#plt.plot(data['date'], data['low'])
#plt.show()

#sys.exit()

Train Start: 259
Train End  : 517
Test  Start: 517
Test  End  : 776
(U+D)/(U+D+E): 35%
(U+D)/(U+D+E): 37%
(U+D)/(U+D+E): 40%
(U+D)/(U+D+E): 43%
(U+D)/(U+D+E): 45%
(U+D)/(U+D+E): 49%
(U+D)/(U+D+E): 51%
(U+D)/(U+D+E): 54%
(U+D)/(U+D+E): 56%
(U+D)/(U+D+E): 60%
(U+D)/(U+D+E): 63%
(U+D)/(U+D+E): 64%
(U+D)/(U+D+E): 66%
p            : 0.4600000000000002
UP   COUNT   : 156
DOWN COUNT   : 189
EVNE COUNT   : 172
(U+D)/(U+D+E): 66%
max: 125.85
min: 99.101
ave: 112.4755
dif: 13.374499999999998


In [4]:
# データ準備
#ここではhigh/lowのtrain/testの４つを整備
#配列的にl_of_sとn_next分の前後にとる
#X = 入力データ
#y = ラベル,教師データ
X_high_train, y_high_train = _load_data(data[['high']].iloc[train_start_count-l_of_s:train_end_count+n_next], \
                                        l_of_s, n_next, True)
X_high_test , y_high_test  = _load_data(data[['high']].iloc[test_start_count-l_of_s:test_end_count+n_next], \
                                        l_of_s, n_next, True)
X_low_train , y_low_train  = _load_data(data[['low']].iloc[train_start_count-l_of_s:train_end_count+n_next], \
                                        l_of_s, n_next, False)
X_low_test  , y_low_test   = _load_data(data[['low']].iloc[test_start_count-l_of_s:test_end_count+n_next], \
                                        l_of_s, n_next, False)

# 正規化
#X_high_train = min_max(X_high_train)
#scaler  = MinMaxScaler( feature_range=(0, 1) )    
#X_high_train = scaler.fit_transform(X_high_train)
#print("行列Xの大きさ:", X_high_train.shape)
#print("行列Xの行数:", X_high_train.shape[0])
#print("行列Xの列数:", X_high_train.shape[1])

#y_high_train = scaler.fit_transform(y_high_train)
#print("行列yの大きさ:", y_high_train.shape)
#print("行列yの行数:", y_high_train.shape[0])
#print("行列yの列数:", y_high_train.shape[1])

X_high_train = (X_high_train-min_value)/(max_value-min_value)
y_high_train = (y_high_train-min_value)/(max_value-min_value)
X_high_test = (X_high_test-min_value)/(max_value-min_value)
y_high_test = (y_high_test-min_value)/(max_value-min_value)
X_low_train = (X_low_train-min_value)/(max_value-min_value)
y_low_train = (y_low_train-min_value)/(max_value-min_value)
X_low_test = (X_low_test-min_value)/(max_value-min_value)
y_low_test = (y_low_test-min_value)/(max_value-min_value)


#####################
# テストパターンの定義 #
#####################
# 活性化関数
activation = ["relu", "tanh", "sigmoid", "hard_sigmoid", "linear", "softmax"]
# 最適化アルゴリズム
optimizer = [SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam]
# 出力層のユニット数
out_dim = [1]
# 隠れ層のユニット数
hidden_dim = [128, 256, 512]
# エポック数
nb_epoch = [5, 10, 100]
# バッチサイズ
batch_size = [5, 10, 30]
# 学習率
learning_rate = [0.001, 0.01, 0.5, 5.0]
# ドロップアウト率
dropout_rate = [0.1, 0.2, 0.5]


# EarlyStopping定義
early_stopping = EarlyStopping(patience=0, verbose=1)

# ニューラルネットの定義
model = KerasClassifier(build_fn=lstm_model, verbose=2)
# テストパターンの設定
param_grid = dict(activation=activation, 
                  Optimizer=optimizer, 
                  out_dim=out_dim,
                  hidden_dim=hidden_dim,
                  nb_epoch=nb_epoch, 
                  batch_size=batch_size,
                  learning_rate=learning_rate,
                  dropout_rate=dropout_rate
                 )
# グリッドサーチ
#grid = GridSearchCV(estimator=model, param_grid=param_grid)
#result = grid.fit(X_high_train, y_high_train)

# ランダムサーチ
# サーチ回数
n_iter_search = 30
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, 
                                   n_iter=n_iter_search, n_jobs=1, verbose=2)
result=random_search.fit(X_high_train, y_high_train, callbacks=[early_stopping])

# 学習
print("High Fit")

# テスト結果表示
print("Best: %f using %s" % (result.best_score_, result.best_params_))
means = result.cv_results_['mean_test_score']
stds = result.cv_results_['std_test_score']
params = result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

sys.exit()




# 2018/1/24 Kitamura Add Start
# 結果のプロット
#high_result = pandas.DataFrame(high_predicted)
#high_result.columns = ['predict']
#high_result['actual'] = y_high_test
#high_result.plot()
#plt.show()
# 2018/1/24 Kitamura Add End

print("Low Fit")
# 学習
X_low_train = X_low_train[ len(X_low_train) % batch_size: ]
low_model.fit(X_low_train, y_low_train, batch_size=batch_size, epochs=100, validation_split=0.0)

# テスト結果表示
low_predicted = low_model.predict(X_low_test)

# 2018/3/3 Kitamura add Start
#正規化
y_low_test = scaler.fit_transform(y_low_test)
# 2018/3/3 Kitamura add End

# 2018/1/24 Kitamura Add Start
# 結果のプロット
low_result = pandas.DataFrame(low_predicted)
low_result.columns = ['predict']
low_result['actual'] = y_low_test
low_result.plot()
plt.show()
# 2018/1/24 Kitamura Add End



Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] batch_size=30, Optimizer=<class 'keras.optimizers.Nadam'>, out_dim=1, dropout_rate=0.5, learning_rate=5.0, hidden_dim=256, nb_epoch=10, activation=sigmoid 
Epoch 1/1
 - 1s - loss: 5899.0085 - acc: 0.0058


C:\Users\kaoru.kitamura\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks.py:497: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.5s remaining:    0.0s


[CV]  batch_size=30, Optimizer=<class 'keras.optimizers.Nadam'>, out_dim=1, dropout_rate=0.5, learning_rate=5.0, hidden_dim=256, nb_epoch=10, activation=sigmoid, total=   3.5s
[CV] batch_size=30, Optimizer=<class 'keras.optimizers.Nadam'>, out_dim=1, dropout_rate=0.5, learning_rate=5.0, hidden_dim=256, nb_epoch=10, activation=sigmoid 
Epoch 1/1
 - 1s - loss: 6283.6112 - acc: 0.0058
[CV]  batch_size=30, Optimizer=<class 'keras.optimizers.Nadam'>, out_dim=1, dropout_rate=0.5, learning_rate=5.0, hidden_dim=256, nb_epoch=10, activation=sigmoid, total=   3.0s
[CV] batch_size=30, Optimizer=<class 'keras.optimizers.Nadam'>, out_dim=1, dropout_rate=0.5, learning_rate=5.0, hidden_dim=256, nb_epoch=10, activation=sigmoid 
Epoch 1/1
 - 2s - loss: 5809.7310 - acc: 0.0058
[CV]  batch_size=30, Optimizer=<class 'keras.optimizers.Nadam'>, out_dim=1, dropout_rate=0.5, learning_rate=5.0, hidden_dim=256, nb_epoch=10, activation=sigmoid, total=   3.5s
[CV] batch_size=5, Optimizer=<class 'keras.optimizers.

Epoch 1/1
 - 3s - loss: 2031.0759 - acc: 0.0000e+00
[CV]  batch_size=5, Optimizer=<class 'keras.optimizers.Adamax'>, out_dim=1, dropout_rate=0.1, learning_rate=0.5, hidden_dim=128, nb_epoch=100, activation=linear, total=   5.3s
[CV] batch_size=5, Optimizer=<class 'keras.optimizers.Adamax'>, out_dim=1, dropout_rate=0.1, learning_rate=0.5, hidden_dim=128, nb_epoch=100, activation=linear 
Epoch 1/1
 - 3s - loss: 1621.9858 - acc: 0.0058
[CV]  batch_size=5, Optimizer=<class 'keras.optimizers.Adamax'>, out_dim=1, dropout_rate=0.1, learning_rate=0.5, hidden_dim=128, nb_epoch=100, activation=linear, total=   6.3s
[CV] batch_size=10, Optimizer=<class 'keras.optimizers.Adagrad'>, out_dim=1, dropout_rate=0.5, learning_rate=0.5, hidden_dim=512, nb_epoch=100, activation=linear 
Epoch 1/1
 - 5s - loss: 4028.3538 - acc: 0.0058
[CV]  batch_size=10, Optimizer=<class 'keras.optimizers.Adagrad'>, out_dim=1, dropout_rate=0.5, learning_rate=0.5, hidden_dim=512, nb_epoch=100, activation=linear, total=   8.0

Epoch 1/1
 - 4s - loss: 5203.5163 - acc: 0.0058
[CV]  batch_size=30, Optimizer=<class 'keras.optimizers.Adagrad'>, out_dim=1, dropout_rate=0.2, learning_rate=0.01, hidden_dim=128, nb_epoch=5, activation=linear, total=   7.3s
[CV] batch_size=5, Optimizer=<class 'keras.optimizers.Adam'>, out_dim=1, dropout_rate=0.5, learning_rate=0.01, hidden_dim=512, nb_epoch=10, activation=linear 
Epoch 1/1
 - 8s - loss: 2379.3569 - acc: 0.0116
[CV]  batch_size=5, Optimizer=<class 'keras.optimizers.Adam'>, out_dim=1, dropout_rate=0.5, learning_rate=0.01, hidden_dim=512, nb_epoch=10, activation=linear, total=  12.3s
[CV] batch_size=5, Optimizer=<class 'keras.optimizers.Adam'>, out_dim=1, dropout_rate=0.5, learning_rate=0.01, hidden_dim=512, nb_epoch=10, activation=linear 
Epoch 1/1
 - 8s - loss: 2504.2075 - acc: 0.0000e+00
[CV]  batch_size=5, Optimizer=<class 'keras.optimizers.Adam'>, out_dim=1, dropout_rate=0.5, learning_rate=0.01, hidden_dim=512, nb_epoch=10, activation=linear, total=  12.4s
[CV] batc

...

 - 74s - loss: 5894.7886 - acc: 0.0058
[CV]  batch_size=10, Optimizer=<class 'keras.optimizers.Adadelta'>, out_dim=1, dropout_rate=0.2, learning_rate=5.0, hidden_dim=512, nb_epoch=5, activation=hard_sigmoid, total= 1.4min
[CV] batch_size=10, Optimizer=<class 'keras.optimizers.Adadelta'>, out_dim=1, dropout_rate=0.2, learning_rate=5.0, hidden_dim=512, nb_epoch=5, activation=hard_sigmoid 
Epoch 1/1
 - 12s - loss: 6276.8821 - acc: 0.0058
[CV]  batch_size=10, Optimizer=<class 'keras.optimizers.Adadelta'>, out_dim=1, dropout_rate=0.2, learning_rate=5.0, hidden_dim=512, nb_epoch=5, activation=hard_sigmoid, total=  19.2s
[CV] batch_size=10, Optimizer=<class 'keras.optimizers.Adadelta'>, out_dim=1, dropout_rate=0.2, learning_rate=5.0, hidden_dim=512, nb_epoch=5, activation=hard_sigmoid 
Epoch 1/1
 - 12s - loss: 5802.5905 - acc: 0.0058
[CV]  batch_size=10, Optimizer=<class 'keras.optimizers.Adadelta'>, out_dim=1, dropout_rate=0.2, learning_rate=5.0, hidden_dim=512, nb_epoch=5, activation=hard_s

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 16.7min finished


Epoch 1/1
 - 12s - loss: 13566.0134 - acc: 0.0039
High Fit
Best: 0.019380 using {'batch_size': 10, 'Optimizer': <class 'keras.optimizers.Adadelta'>, 'out_dim': 1, 'dropout_rate': 0.1, 'learning_rate': 0.001, 'hidden_dim': 128, 'nb_epoch': 10, 'activation': 'relu'}
0.000000 (0.000000) with: {'batch_size': 30, 'Optimizer': <class 'keras.optimizers.Nadam'>, 'out_dim': 1, 'dropout_rate': 0.5, 'learning_rate': 5.0, 'hidden_dim': 256, 'nb_epoch': 10, 'activation': 'sigmoid'}
0.000000 (0.000000) with: {'batch_size': 5, 'Optimizer': <class 'keras.optimizers.Adadelta'>, 'out_dim': 1, 'dropout_rate': 0.5, 'learning_rate': 0.5, 'hidden_dim': 256, 'nb_epoch': 100, 'activation': 'sigmoid'}
0.000000 (0.000000) with: {'batch_size': 30, 'Optimizer': <class 'keras.optimizers.Adamax'>, 'out_dim': 1, 'dropout_rate': 0.5, 'learning_rate': 0.001, 'hidden_dim': 128, 'nb_epoch': 10, 'activation': 'sigmoid'}
0.000000 (0.000000) with: {'batch_size': 30, 'Optimizer': <class 'keras.optimizers.Adamax'>, 'out_dim'

SystemExit: 

C:\Users\kaoru.kitamura\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#high/lowの予想最大/最小値のグラフ（小さくてわからない）
#result = pandas.DataFrame(high_predicted)
#result.columns = ['high_predict']
#result['low_predict'] = low_predicted
#result.plot()
#plt.show()

#
pre_check = []
temp_check = []
temp_close_open = []
temp_close_open_up_win = []
temp_close_open_up_lost = []
temp_close_open_down_win = []
temp_close_open_down_lost = []

#使わないデータも保持しているが
#予想したHighの最大値とLowの最小値を起点の日の翌日のOpenの
#と比較し、最初sに決定した差よりも大きい場合は上がり・下がりと
#判断する
#ただし、同時超えた場合はどちらが先に上がるか不明なためカウントしていない
#もしかすると
for i in range(len(low_predicted)):
    high_temp = high_predicted[i] * diff_value + average_value
    low_temp  = low_predicted[i] * diff_value + average_value
    open_temp = data.loc[i+test_start_count+1, 'open'] * diff_value + average_value
    close_temp = data.loc[i+test_start_count, 'close'] * diff_value + average_value

    if high_temp - close_temp >= check_treshhold and \
        close_temp - low_temp < check_treshhold:
        pre_check.append(1)
        temp_check.append(high_temp - open_temp)
        temp_close_open.append(abs(close_temp - open_temp))

    elif close_temp- low_temp >= check_treshhold and \
        high_temp - close_temp < check_treshhold:
        pre_check.append(-1)
        temp_check.append(low_temp - open_temp)
        temp_close_open.append(abs(close_temp - open_temp))
    #elif high_temp - open_temp >= check_treshhold and \
    #    open_temp- low_temp >= check_treshhold:
        
    #    if high_temp - open_temp > open_temp- low_temp:
    #        pre_check.append(1)
    #        temp_check.append(high_temp - open_temp)
    #    elif high_temp - open_temp < open_temp- low_temp:
    #        pre_check.append(-1)
    #        temp_check.append(low_temp - open_temp)
    #    else:
    #        pre_check.append(0)
    #        temp_check.append(0)

    #if high_temp - open_temp >= check_treshhold and \
    #    open_temp - low_temp < check_treshhold and \
    #    close_temp - open_temp <= up_c_o_diff and \
    #    close_temp - open_temp >= down_c_o_diff:
    #    pre_check.append(1)
    #    temp_check.append(high_temp - open_temp)

    #elif open_temp- low_temp >= check_treshhold and \
    #    high_temp - open_temp < check_treshhold and \
    #    close_temp - open_temp <= up_c_o_diff and \
    #    close_temp - open_temp >= down_c_o_diff:
    #    pre_check.append(-1)
    #    temp_check.append(low_temp - open_temp)

    else:
        pre_check.append(0)
        temp_check.append(0)

up_ok_count = 0
up_ng_count = 0
up_ev_count = 0
down_ok_count = 0
down_ng_count = 0
down_ev_count = 0
high_win = numpy.array([])
high_lost = numpy.array([])
low_win  = numpy.array([])
low_lost  = numpy.array([])

for i in range(len(pre_check)):
    if pre_check[i] == 1:
        if up_down[i] == pre_check[i]:
            up_ok_count += 1
            high_win = numpy.append(high_win, numpy.array(temp_check[i]))
        elif up_down[i] != pre_check[i] and up_down[i] == -1:
            up_ng_count += 1
            high_lost = numpy.append(high_lost, numpy.array(temp_check[i]))
        else:
            up_ev_count += 1
            
    elif pre_check[i] == -1:
        if up_down[i] == pre_check[i]:
            down_ok_count += 1
            low_win = numpy.append(low_win, numpy.array(temp_check[i]))
        elif up_down[i] != pre_check[i] and up_down[i] == 1:
            down_ng_count += 1
            low_lost = numpy.append(low_lost, numpy.array(temp_check[i]))
        else:
            down_ev_count += 1


print('==========')
print('UP:')
print(' WIN  :' + str(up_ok_count))
print(' LOST :' + str(up_ng_count))
print(' DRAW :' + str(up_ev_count))
print(' WIN RATE :' + str(math.floor((up_ok_count/(up_ok_count+up_ng_count))*100)) + '%')
print('DOWN:')
print(' WIN  :' + str(down_ok_count))
print(' LOST :' + str(down_ng_count))
print(' DRAW :' + str(down_ev_count))
print(' WIN RATE :' + str(math.floor((down_ok_count/(down_ok_count+down_ng_count))*100)) + '%')
#print('---------')
#print('UP ')
#print('  WIN  :' + str(high_win.mean()) + ' ' + str(high_win.std()))
#print('  LOST  :' + str(high_lost.mean()) + ' ' + str(high_lost.std()))
#print('DOWN ')
#print('  WIN  :' + str(low_win.mean()) + ' ' + str(low_win.std()))
#print('  LOST  :' + str(low_lost.mean()) + ' ' + str(low_lost.std()))


print(str(math.floor(time.time() - starttime)) + "s")